<a href="https://colab.research.google.com/github/UdaraChamidu/Generative-AI/blob/main/Langchain_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG application with Langchain and HuggingFace LLM

In [92]:
# Install the necessary packages
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q # an embedding model

In [93]:
import os
from google.colab import userdata

# LLM

In [94]:
from langchain_community.llms import HuggingFaceHub

# Initialize the HuggingFace llm
llm = HuggingFaceHub(
      repo_id = "mistralai/Mistral-7B-v0.1",   # copied from hugging face site
      model_kwargs = {"temperature":0.1, "max_length":500}, # max length of output = 500
      huggingfacehub_api_token = userdata.get('HUGGINGFACE_API_KEY'))

# Embedding Model

In [95]:
pip install -U langchain-huggingface


In [96]:
# goto sentence transform documentation and find a pre trained model

from langchain.embeddings import HuggingFaceEmbeddings

# 1. Load a pretrained Sentence Transformer model
embeding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Output Parser

In [97]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

Load PDF

In [98]:
!pip install pypdf -qU

In [99]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("Eye Disease Classification Using DL.pdf")

docs = loader.load()

In [100]:
docs[0]

Document(metadata={'producer': 'MiKTeX pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T20:43:06-05:00', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'moddate': '2023-07-20T20:43:06-05:00', 'trapped': '/False', 'ptex.fullbanner': 'This is MiKTeX-pdfTeX 4.11.0 (1.40.24)', 'rgid': 'PB:372717102_AS:11431281177632256@1690552117401', 'source': 'Eye Disease Classification Using DL.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/372717102\nEye Disease Classiﬁcation Using Deep Learning Techniques\nConference Paper · July 2023\nDOI: 10.48550/arXiv.2307.10501\nCITATIONS\n12\nREADS\n2,501\n5 authors, including:\nTareq Babaqi\nBinghamton University\n15 PUBLICATIONS\xa0\xa0\xa041 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nSaif Al-Nimer\nBinghamton University\n3 PUBLICATIONS\xa0\xa0\xa014 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content

In [101]:
len(docs)

7

# Split DOcuments into chunks

In [102]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [103]:
len(splits)

58

### Create Vector Store and *Retriever*

In [104]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embeding_model)

In [105]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define Prompt Template

In [106]:
from langchain.prompts import ChatPromptTemplate

# Define prompt template

# provided instruction
# question
# context
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""

prompt=ChatPromptTemplate.from_template(template)

In [107]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.  \n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'), additional_kwargs={})])

### Chain Retriever and Prompt Template with LLM

In [108]:
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

# context = chunks from retriever
# question = the question we give when we run
# after that all those things send to the prompt => llm => output_parser

#### Invoke RAG Chain with Example Questions



In [109]:
response = chain.invoke(input = "what is EfficientNet ?")
print(response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-v0.1